# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium


# Import API key
from api_keys import geoapify_key
print(geoapify_key)


9032fed14dad4d6190ac9a7981404541


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,enewetak,11.3474,162.3373,27.41,79,96,7.63,MH,1713371149
1,1,port-aux-francais,-49.3500,70.2167,3.84,98,100,11.83,TF,1713371149
2,2,east london,-33.0153,27.9116,22.52,73,0,3.09,ZA,1713371152
3,3,bilibino,68.0546,166.4372,-22.90,98,53,2.01,RU,1713371152
4,4,margaret river,-33.9500,115.0667,18.47,48,0,5.54,AU,1713371152


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x="Lng",
    y="Lat", 
    geo = True, 
    tiles="OSM",
    frame_width=700, 
    frame_height=600,
    size="Humidity", 
    scale=1, 
    color="City"
    )

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down the DataFrame to find cities with ideal weather conditions
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,east london,-33.0153,27.9116,22.52,73,0,3.09,ZA,1713371152
80,80,rakops,-21.0167,24.3333,26.56,44,0,3.42,BW,1713371157
83,83,holualoa,19.6228,-155.9522,21.59,46,0,2.06,US,1713371157
150,150,port elizabeth,-33.9180,25.5701,25.61,64,0,2.06,ZA,1713371162
172,172,gazojak,41.1875,61.4036,24.14,21,0,3.84,TM,1713371172


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
2,east london,ZA,-33.0153,27.9116,73,
80,rakops,BW,-21.0167,24.3333,44,
83,holualoa,US,19.6228,-155.9522,46,
150,port elizabeth,ZA,-33.9180,25.5701,64,
172,gazojak,TM,41.1875,61.4036,21,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
limit = 5
apiKey = geoapify_key
params = {
    "limit":limit,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    params["categories"] = f'accommodation.hotel'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
east london - nearest hotel: No hotel found
rakops - nearest hotel: Rakops River Lodge
holualoa - nearest hotel: Kona Hotel
port elizabeth - nearest hotel: Waterford Hotel
gazojak - nearest hotel: Жайхун
tazacorte - nearest hotel: App Leyma
emporia - nearest hotel: Budget Host Inn Emporia
port hedland - nearest hotel: The Esplanade Hotel
kailua-kona - nearest hotel: Kona Seaside Hotel
ha tinh - nearest hotel: Hong Chau Nha Nhgi
lompoc - nearest hotel: Red Roof Inn Lompoc
el hamma - nearest hotel: No hotel found
kruisfontein - nearest hotel: No hotel found
uskudar - nearest hotel: فندقي


,City,Country,Lat,Lng,Humidity,Hotel Name
2,east london,ZA,-33.0153,27.9116,73,No hotel found
80,rakops,BW,-21.0167,24.3333,44,Rakops River Lodge
83,holualoa,US,19.6228,-155.9522,46,Kona Hotel
150,port elizabeth,ZA,-33.9180,25.5701,64,Waterford Hotel
172,gazojak,TM,41.1875,61.4036,21,Жайхун


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    x="Lng",
    y="Lat", 
    geo = True, 
    tiles="OSM",
    frame_width=700, 
    frame_height=600,
    size="Humidity", 
    scale=1, 
    color="City",
    hover_cols = ["Hotel Name","Country"]
    )

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)